## DESE61003 - Audio Experience Design
# Week 7 - Real-Time Sound Processing

In [ ]:
from packaging.version import parse as parse_version
try:
    import disiple
    if parse_version(disiple.__version__) < parse_version('0.6.0'):
        raise ImportError('Outdated version')
except ImportError:
    %pip install -q --upgrade disiple

In [ ]:
from disiple.signals import AudioSignal, Spectrum, PowerSpectrum, Spectrogram, TimeSignal
import numpy as np
import ipywidgets as widgets
from dataclasses import dataclass
from collections import deque
from enum import Enum, auto

In this session, we'll learn about block-based processing, which is how (pseudo) real-time processing is achieved. We will cover a number of audio effects, as these are a fun way to introduce the concept and their auditory output makes it easy to diagnose problems.

### 1. Block-based processing

The concept of block-based processing is easy: an incoming signal get buffered into small blocks, which are subsequently processed. For audio effects, the output of this process is another audio buffer of the same size. The size of these blocks (or buffers) is an important trade-off that needs to be made.

On the one hand, a buffer won't be processed until it's full, so it imposes a lower limit on the time between an incoming sample and the playback of its processed result, the so-called _latency_. This latency needs to be kept low to maintain the perception of real-time processing, i.e. before the delay between key presses or parameter changes becomes so large you notice. How much time exactly can pass before this delay becomes noticeable depends on the signal, listener and other factors, but it is generally assumed that delays up to 15-20 milliseconds are barely noticeable.

On the other hand, the block size also determines how much time is available for the processing itself. While the earlier processed block is played back, the output samples for the current block need to be calculated. This calculation needs to be done before the already processed block has finished playback, otherwise the constant stream of output samples gets interrupted, which we can hear as "crackling" sounds. It might seem at first that the size of the block is not very relevant for the available processing time. After all, if you double the block size (and therefore the available processing time), you have twice as many samples to process. One thing to realise is that the processing time does not necessarily increase linearly with the number of samples in a buffer. Certain computational techniques such as variable reuse and vector processing make it more efficient to process a whole bunch of samples together, rather than all of them individually.

What's even more important, is that due to the multi-tasking nature of general computing operating systems, the processing power available for our audio effects is not constant, nor guaranteed. Suppose you are not doing anything else on your computer apart from processing an audio stream. The amount of computation required to process a single sample requires only a couple of percents of what the processor can provide, so you decide to use a very small block size and everything runs smoothly. Then suddenly, your email client running in the background decides to check whether a new email has arrived. This requires little computing power too and therefore isn't noticeable to you, the user of the computer. Overall, the constant stream of output samples won't be affected either, since there is plenty of power to process audio and check emails at the same time. However, just when the automatic email check started, the processor had to make a scheduling decision, checking which resources were free and how to spread its tasks. This made processing the single sample at that moment take five times longer than usual. This doesn't necessarily cause any long term impact, there's plenty of power for the next samples to catch up, but because of the small block size, that particular sample needed to be ready for sending to the speakers. Having missed that opportunity, the output stream was interrupted and we hear a brief click in our speakers, even though our processor load never reached double figures. This illustrates another benefit of large block sizes: the larger the buffer, the more robust it is against such worst-case scenarios. The inherently variable processing time _per sample_ is averaged over all samples in the buffer, and it's this _average_ time that determines whether a block is processed fast enough to avoid interrupting the output stream. So even if the (average) processing time increases linearly with the number of samples, larger buffers still have the benefit of smoothing out variations in processing time.

Splitting a signal into blocks for processing is somewhat related to splitting a signal into frames, but there are two key differences. First, the size of a block should not be considered as a user-controllable parameter. While is true that in many audio drivers a specific block size can be requested, and this request will often be honoured, the block size is not guaranteed to be reliable or even constant. Second, blocks are always non-overlapping. Therefore if a specific frame size and/or step size is necessary for further processing, additional measures need to be taken, which we'll cover in the remainder of this notebook.

### 2. Setting up an audio effect framework: the effect of doing nothing

For now, we will limit ourselves to simulating real-time processing by taking an audio file, chopping it into blocks and feeding these blocks in turn into a block-processing function. This way, we don't need to worry about configuring recording equipement (e.g. a mic built into your computer) or overloading the processor such that samples are dropped from the output. Therefore we start by loading some test audio files.

In [ ]:
violin_signal = AudioSignal('../data/Violin.arco.ff.sulG.A4.mono.wav')
cat_person = AudioSignal('../data/Townhouse_Woods_-_Cat_Person.wav')

To create an audio effect, we need to define a block-processing function, which is any function that takes in a NumPy array of samples and return another sample array of equal length. Therefore the easiest function is the one that simply passes the input array as its output, like the one below.

Note: You can see that the function takes another input argument too, which we've named `_` here. Naming a variable `_` signifies that we won't actually use it, but a block-processing function needs to have a second argument to be compatible with the code that calls it. You can ignore its presence for now, we'll discuss its use in a later section.

In [ ]:
def passthrough(input_buffer, _):
    return input_buffer

Such block-processing function can then be used in combination with the `AudioSignal.from_processing_blocks()` class method in `disiple`. A class method is also known as an _alternative constructor_ because it has the same behaviour as a regular constructor, only with a sligthly different syntax. Both create a new object of a certain class, `AudioSignal` in this case. Instead of calling `AudioSignal()` for a regular constructor, we need to call `AudioSignal.from_processing_blocks()`. Its complete mode signature is `AudioSignal.from_processing_blocks(input_signal, block_size, processing_fn)`. We therefore pass our `passthrough` processing function together with any `input_signal` to create a new `AudioSignal`. The last required parameter is `block_size`, and because we should be creating processing functions that produce the same output regardless of the exact block size, it is good practice to test with a couple of different values, like below.

In [ ]:
print('Input signal')
violin_signal.play()
for block_size in [256, 512, 1024]:
    output_signal = AudioSignal.from_processing_blocks(violin_signal, block_size, passthrough)
    print('Output for block size {}'.format(block_size))
    output_signal.play()

Listening to the results, we can tell that the processing function indeed does nothing, and more importantly that the output is the same for all block sizes.

### 3. Adding a user-controlled parameter: a slightly more interesting gain effect

Now that we know how to create and test a block-processing function, we can move on to create audio effects that actually do something. Barely more interesting than doing nothing, but an obligatory first trial (the "hello world" of audio effects) is the creation of a volume control. You should already be able to do this based on what we've seen so far.

#### Activity 1

Create an audio effect that halves the volume (i.e. amplitude, not perceptual loudness) of a signal and test it like we did above. Remember that an input buffer is a NumPy array, so you can use all its mathematical functionality to modify it into the desired output.

In [ ]:
# your code here

Hard-coding the level of volume reduction severely reduces the usefulness of our volume control effect. Ideally, we'd like to expose the volume level as a user-controllable parameter that can be modified outside the actual block-processing function. Not only would we like to set this level at the start of the audio stream we are processing, we'd also like it to respond to changes we make to this parameter while the stream is playing, i.e. achieve a dynamic response like we would twist a physical volume button in real time.

To do so, we can replace the hardcoded value by a variable defined elsewhere and read this value every time the block-processing function is called such that the latest value for the volume control is always used. To avoid straying too far from audio processing, we are not going to simulate dynamic changes to a parameter value while the audio is playing, only change its initial value. The theory behind it, though, is that a parallel thread checks for movements made in a GUI or external hardware (e.g. a virtual or hardware knob being turned) and synchronises them with the corresponding changes to the volume variable stored in a centrally accessible location.

In theory, this centrally accessible location could be anywhere, but it is good practice to keep all code used for the audio effect together and avoid relying on global variables. An elegant way to accomplish this in python is to store parameter data as a [`dataclass`](https://docs.python.org/3/library/dataclasses.html) and use this `dataclass` as a storage object that is passed around between calls to the block-processing function.

A dataclass is defined using the same `class` syntax as a regular class, but with a `@dataclass` statement (known as a _decorator_) added to the line above. The body of the class then lists all its properties (here parameters), declared as `property_name: data_type`. The `dataclass` for this introductory examples contains only one property: `volume` which is of type `float`.

In [ ]:
@dataclass
class ParameterData:
    volume: float

To instantiate an object containing actual data from a `dataclass` definition, we call it like a constructor for any other class, with the class properties as input arguments, here `volume=0.3`.

In [ ]:
parameters = ParameterData(volume=0.3)

As a shortcut, you can also drop the names of the input arguments and list them in the order the `dataclass` members are defined (obviously not relevant when you only have one member).

In [ ]:
parameters = ParameterData(0.3)

Once instantiated, the property values can be read (and overwritten) using so-called _dot-notation_.

In [ ]:
parameters.volume

Such `dataclass` objects can be passed as a fourth, optional argument to `AudioSignal.from_processing_blocks()`, whose type signature becomes `AudioSignal.from_processing_blocks(input_signal, block_size, processing_fn, parameter_data)`. This ensures that the `parameter_data` object gets passed to every invocation of the block-processing function as its second argument. The expected type signature for any block-processing function is therefore `processing_fn(input_buffer, parameter_data)` (but the second argument can be left unused as we saw previously).

#### Activity 2

Combining all this information, create a volume control effect whose volume level is set by a `dataclass` object. Test it using multiple block sizes and verify the output is always the same.

In [ ]:
# your code here

### 4. Getting serious: a distortion effect

What you've learned this far is already sufficient to create many interesting effects. The only thing you need to know is what the underlying mathematical operations are to achieve a certain effect. An interesting, but not too complicated effect to start with is distortion. In its most basic version, it is a non-linear curve that is applied to each sample in the time domain. For classical distortion effects, these curves are more strongly non-linear around the signal extrema (-1 and 1) than around zero, but you could come up with any transformation of samples you like, as long as the result lies within the domain [-1, 1].

Because of the stronger non-linear effect towards -1 and 1, a typical parameter on a distortion effect is the "drive", which is simply a volume control that allows to boost quiet signals into the non-linear saturation zone near +/- 1, before a curve is applied. This process is known as "overdriving" the samples. If omitted, quiet signals stay within the linear range of the curve, making the distortion only perceptible for loud signals (which could be wanted behaviour).

Since a distorted signal (even without "drive" boost) will be perceptually louder than the unprocessed signal, which is not always desired, another volume control can be added after applying the non-linear curve. This parameter is often called "post-gain" (and is mostly configured to _reduce_ volume, as opposed to the drive which is always set to _boost_ the volume).

Another parameter commonly seen on distortion effects is "blend" or "dry/wet", which gives the option to mix the original signal ("dry") with the distorted signal ("wet") to get a more gentle sound.

Finally, the exact shape of the non-linear curve can also be considered a parameter (though not always user-controllable). In the [Audio Developer Conference](https://audio.dev/) talk ["50 Shades of Distortion"](https://www.youtube.com/watch?v=oIChUOV_0w4), a large number of curves is discussed. Most of them are also shown in the graphical calculator embedded below.

In [ ]:
from IPython.display import IFrame
IFrame(src='https://www.desmos.com/calculator/cboncgvvfs', width="100%", height="500")

#### Activity 3

Create a distortion effect with multiple parameters and test it using multiple block sizes. To summarise, the signal flow of a distortion effect is:

- boost the volume of the incoming samples according to the "drive" parameter
- apply a non-linear curve to the overdriven samples
- mix the input signal with the processed samples according to the "blend" parameter
- reduce the volume of the result according to the "post-gain" parameter

Don't try to implement these steps all at once, but test them step by step. The most natural order would be to start by only applying a non-linear curve to the input, then adding a drive parameter. Finally, you can add blend and post-gain or experiment with different curves.

To get you started, a dataclass for all parameters is given below. Note that the dataclass provides a parameter to switch between different limiter curves, called "mode". It is a discrete parameter, as opposed to the continuous values of `drive`, `blend` and `post_gain`. A fixed number of choices like this can be represented by integers (0 for the first option, 1 for the second, etc.), but a more elegant option is to define an [_enumeration_ (_enum_ for short)](https://docs.python.org/3/library/enum.html) of all possible options. This create a datatype of named constants (therefore the values are written in uppercase by convention), which are easier to read than arbitrary integers. They also protect against using undefined values (e.g. using mode "23" when only 0-9 are defined). You can assign the enum values to variables and check their equality like you would do with any other type, e.g. `mode == DistortionMode.HARD_LIMITER` (rank operators like `<` or `>=` are not available though). The enumerated values correspond to the $f_x(t)$ curves defined in the Desmos graphical calculator, but you obviously don't need to implement them all (or even use the `mode` parameter). Finally, some functions that could come in handy are [`trigonometric functions`](https://numpy.org/doc/stable/reference/routines.math.html), [`np.clip`](https://numpy.org/doc/stable/reference/generated/numpy.clip.html), [`np.sign`](https://numpy.org/doc/stable/reference/generated/numpy.sign.html) and [`scipy.special.erf`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.erf.htm).

In [ ]:
class DistortionMode(Enum):
    HARD_LIMITER = auto()
    TANH_LIMITER = auto()
    ERF_LIMITER = auto()
    ABS_LIMITER = auto()
    ROOTSQUARE_LIMITER = auto()
    ARCTAN_LIMITER = auto()
    GUD_LIMITER = auto()
    SIGN_LIMITER = auto()
    SINE_FOLDOVER = auto()

@dataclass
class DistortionParameters:
    mode: DistortionMode
    drive: float
    blend: float
    post_gain: float

In [ ]:
# your code here

### 5. Remembering the past: a delay effect

One of the reasons why a distortion effect is relatively easy to implement is that each sample is processed independently. The corresponding output is influenced by the parameters settings, but not by other samples. We call such type of effect _stateless_. Ignoring the realtime constraints, you could process each sample in random order and still arrive at the same output. Its opposite are _stateful_ effects, which do depend on earlier samples. An example of a stateful effect is a delay.

Conceptually, a delay effect is rather simple. An incoming sample is mixed with an earlier sample and sent out for playback. The amount of time elapsed between the current time and the time of the earlier sample determines the character of the effect and is known as the _delay time_. The technical implementation of a delay effect is less straightforward though. In block-based processing, earlier samples are not (all) readily available. Surely, given a large enough buffer and a small enough delay time, some of the required earlier samples can be read from the current block, but all earlier samples of the first sample in a block are by definition processed in the previous block and therefore no longer available.

We thus need an additional memory structure, that ensure we always keep the required number of past samples for a given delay time. Once the earlier sample corresponding to the current sample for a given delay times is processed, we won't ever need to use it again, so it can be forgotten. We don't need to keep all of the past sample in memory, only a buffer of fixed length which is constantly updated: for every new "current" sample coming in, the old "earlier" sample can be forgotten. Such a memory structure is generally known as a [first-in-first-out or FIFO](https://en.wikipedia.org/wiki/FIFO_(computing_and_electronics)) buffer. For audio processing, it is common to call this concept a [circular or ring buffer](https://en.wikipedia.org/wiki/Circular_buffer).

A data structure that is readily available in the Python standard library and that can be used as an implementation for the abstract concept of "circular buffer" is a [`deque`](https://docs.python.org/3/library/collections.html), short for double-ended queue. For time-domain processing, we want to initialise the contents of a deque as all zeros. After all, there is a cold start problem, the first samples of a stream don't have any earlier samples to read. Initialising the buffer to all zeros corresponds conceptually to prepending silence to the audio stream, which is what we want. You can remove and read the oldest sample in a `deque` by calling `deque.popleft()` and store a new value by calling `deque.append()`. If you alternate calls to `popleft()` and `append()`, the deque's length will remain constant at the length of its initialisation. For efficiency reasons, it helps if we specify this constant length as the constructor argument `maxlen=`.

In [ ]:
circ_buffer = deque(np.zeros(4096), maxlen=4096)

This circular buffer needs to be kept up to date by every invocation of the block-processing function, and needs to be passed between subsequent calls. We already have a mechanism in place to do so, passing a `dataclass` like we did to share parameter values. This `dataclass` will store the state of our audio effect at any given time.

In [ ]:
@dataclass
class DelayState:
    circ_buffer: deque

#### Activity 4

The size of the circular buffer corresponds to the delay time. Can you figure out what the buffer size needs to be for a given delay time (in seconds)? What other info do you need to calculate this? Do any extra properties need to be added the `dataclass` to keep track of this?

_your answer here_

#### Activity 5

Implement and test a delay effect that mixes a signal with a copy of itself delayed by a constant amount (given at the start and assumed not to change over the duration of the audio stream). Hint: it is easiest to create the output sample-by-sample and not to use any vector processing in this case.

In [ ]:
# your code here

#### Activity 6

It has been strategically left unmentioned so far, but the earlier sample that is mixed with the current sample can actually be either an earlier _input_ sample or an earlier _output_sample. The first option creates a delay effect without feedback and the second option a delay effect with feedback. Now implement the other of the two (most likely you will have done the version without feedback first). Can you hear any difference? It becomes easier to distinguish the two for longer delay times, 0.5 seconds and more.

In [ ]:
# your code here

### 6. Going further

If you want to delve deeper into the topic, you can start by making small extensions to the delay effect, such as adding parameters to control the balance between the current and earlier sample or to unify both delay effects into one that can switch between feedback on or off. After that, you can move on to other effects such as tremolo (amplitude modulation, which is stateful because you need to keep track of the modulating signal between blocks to avoid discontinuities) or more advanced delay-based effect such as flanging (a dynamic delay with periodically changing delay time, so you'll need to keep track of the delay time too). An extensive resource is the book ["DAFX: Digital Audio Effects"](https://onlinelibrary.wiley.com/doi/book/10.1002/9781119991298), edited by Udo Zölzer.